<a href="https://colab.research.google.com/github/RaincallerMei/Galaxy-Zoo-Solution/blob/main/Astro_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import scipy as sp
import skimage as ski
import pandas as pd

ModuleNotFoundError: No module named 'pylearn2'

In [4]:
from theano import *
import theano.tensor as T
#import pylearn2 as pylearn

ModuleNotFoundError: No module named 'theano'